In [11]:
import importlib

ModuleNotFoundError: No module named 'losses_PT'

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
#%% Training Loop 1 (dataloader) [not important]
j = 0
for sample in data_loader:    
    sample = sample.cuda()
    optimizer.zero_grad()   # zero the gradient buffers
    output = tan_m(net(sample[0]))
    loss1 = L(output[0:1], output[1:2], output[2:3])
    loss1.backward()

    optimizer.step()   #%%
    if j%500==0:
        print(loss1.cpu())
        # pdb.set_trace()
    j= j+1

In [ ]:
#%% Alternate Training Loop (functional) [not important ]

tdc =  test_data(temp)
j = 0
net.train()
for _ in range(1):
    sample = torch.from_numpy(tdc.get_data_Triplet())
    sample = sample.cuda()
    optimizer.zero_grad()   # zero the gradient buffers
    output = tan_m(net(sample))
    loss1 = L(output[0:1], output[1:2],output[2:3])
    loss1.backward()
    if j%2000==0:
        print(loss1.cpu())
        # pdb.set_trace()
    optimizer.step()   #%%
    j= j+1

In [ ]:
#%% SimCLR training loop (functional loop not efficient)

optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)


L= NT_Xent(batch_size=2, temperature=0.8, world_size=1)
L= L.cuda()

tdc =  test_data(temp) # temp should be one pickle file
j = 0
net.train()


for _ in range(10000):
    s1, s2 = torch.from_numpy(tdc.get_SIMCLR_data()[0]), torch.from_numpy(tdc.get_SIMCLR_data()[1]) 
    s1, s2 =  s1.cuda(), s2.cuda()
    optimizer.zero_grad()   # zero the gradient buffers
    o1, o2 = net(s1),  net(s2)
    loss1 = L(o1, o2)
    loss1.backward()
    optimizer.step() 
    if j%500==0:
        print(loss1.cpu())
        # pdb.set_trace()
    j= j+1

In [ ]:
class DummyDatasetMADA(data.Dataset):
    def __init__(self, window_size, n_channel, n_classes, n_samples):
        self.X = np.random.rand(n_samples, n_channel, window_size, 1)
        self.y = np.random.randint(n_classes, size=(n_samples,))

    def __getitem__(self, index):
        return self.X[index], self.y[index]

    def __len__(self):
        return self.X.shape[0]

In [1]:
#%% Test the data-loader section
s1, s2 = next(iter(data_loader))
xx = torch.cat((s1[0], s2[0]), 0)
xx.shape

NameError: name 'data_loader' is not defined

In [2]:
#%% Test the data-loader section by visualization
s1,s2= next(iter(data_loader))
s1_trx = trx(s1)
for i in range (7):
    plt.imshow(np.moveaxis((s1[0][i,:,0,:,:]).cpu().numpy(), 0,2), vmin=0., vmax=1.)
    plt.figure()
    plt.imshow(np.moveaxis((s2[0][i,:,0,:,:]).cpu().numpy(), 0,2), vmin=0., vmax=1.)
    plt.figure()

NameError: name 'data_loader' is not defined

In [ ]:
s1,s2= next(iter(data_loader))
# s1_trx = trx(s1)
for i in range(4):
    l1 = np.moveaxis((s1[0][i,:,0,:,:]).cpu().numpy(), 0,2)
#     fig1 = plt.gcf()
#     plt.plot()
#     plt.imshow(l1)
#     fig1.savefig('myimage'+ str(i) + '.png', format='png', dpi=1200)
#     plt.figure()
#     fig2 = plt.gcf()
    l2 = np.moveaxis((s2[0][i,:,0,:,:]).cpu().numpy(), 0,2)
#     plt.plot()
    plt.imshow(l2)
#     fig2.savefig('myimage0'+ str(i) + '.png', format='png', dpi=1200)
#     plt.figure()
#     plt.imshow(np.moveaxis((s2[0][i,:,0,:,:]).cpu().numpy(), 0,2), vmin=0., vmax=1.)
#     plt.figure()
#     print(get_key(LabelDict, s2[0][i]))

In [ ]:
#https://theailearner.com/2018/10/15/creating-video-from-images-using-opencv-python/
def video_save(sp_images, size = (300, 300)):
    out = cv2.VideoWriter('project.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)    
    for i in range(len(sp_images)):
        out.write(sp_images[i])
    out.release()

In [ ]:
#%% create a binary pickle file for label_info
path = "../label_info.pkl"
def save_dict(dict, path):
    f = open(path,"wb")

    pickle.dump(dict,f)
    f.close()
    
save_dict(label_info, path)
def load_dict(path):

    with open(path, 'rb') as f:
        x = pickle.load(f)
    
    return x

ss =  load_dict(path)

In [ ]:
s1 = next(iter(data_loader))

with torch.no_grad():
    net.train()
    s1 = trx(s1)
    s1= s1[0].cuda()
    opt = net(s1)
    opt0 = net(s1[0:1])
    
print(opt[1])
print(opt0)


In [ ]:
#%% some data visualization
plt.imshow(np.moveaxis((sample[0,:,0,:,:]).cpu().numpy(), 0,2), vmin=0., vmax=1.)
plt.imshow(np.moveaxis((sample[0,:,0,:,:]).cpu().numpy(), 0,2), vmin=0., vmax=1.)
plt.imshow(temp[7][0][0])

In [56]:
loss = InfoNCE()
loss(ot[0:1], ot[1:2], ot[2:])

tensor(1.9507, device='cuda:0')

In [40]:
#%% Analysis
cos = nn.CosineSimilarity()
pos = torch.exp(cos(ot[0:1], ot[1:2])/0.05)
negs = torch.exp(cos(ot[0:1], ot[2:])/0.05)
neg = torch.sum(negs)
denominator = pos+neg
print(pos)
print(negs)
print(denominator)
-torch.log(pos/denominator)

tensor([4.4990e+08], device='cuda:0')
tensor([4.3785e+08, 4.2815e+08, 4.2169e+08, 4.2480e+08, 4.2780e+08, 4.1624e+08],
       device='cuda:0')
tensor([3.0064e+09], device='cuda:0')


tensor([1.8995], device='cuda:0')

In [ ]:
def get_label_name(idx , pos, gt_id, gt_name): # send [idx[0]]
    return gt_name[np.where(gt_id == gt_id[gt_id<idx[pos].numpy()][-1])[0][0]]

def verify_supset(pos, idx, s1, gt_id, gt_name, fr = 1 ):
    print(get_label_name(idx[0], pos, gt_id, gt_name))
    plt.imshow(np.moveaxis((s1[0][pos,:,fr,:,:]).cpu().numpy(), 0,2), vmin=0., vmax=1.)
    plt.show()
    return


LabelDict= {'sitting':0, 'standing':1, 'lying_d':2, 'lying_u':3, 'walking':4, 'push_up':5, 'object_walk':6, 'object_pick':7, 'hand_wave':7, 'leg_exer':9, 'what':10}
    

def get_key(my_dict, val):
    for key, value in my_dict.items():
         if val == value:
             return key
    return "key doesn't exist"

In [ ]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

my_net.blocks[0].activation.register_forward_hook(get_activation('l1'))

In [ ]:
#%% Test the data-loader section by visualization
s1 = next(iter(data_loader))
s1_trx = trx(s1)

for i in range (7):
    plt.imshow(np.moveaxis((s1_trx[0][i,:,0,:,:]).cpu().numpy(), 0,2), vmin=0., vmax=1.)
    plt.figure()

In [ ]:
sample = s1_trx[0][4:5]

sample = sample.cuda()

with torch.no_grad():
    output = my_net(sample)

In [ ]:
s1_trx[0][5].shape
activation['l1'][0].shape

In [ ]:
for i in range (7):
    plt.imshow(np.moveaxis((activation['l1'][0][i,0:1,:,:]).cpu().numpy(), 0,2), vmin=0., vmax=1.)
    plt.figure()

In [ ]:
label_info ={}

label_info[files[0]] = {'gt_id': np.int16([0,     30*30 ,      60*30,     95*30,      125*30,    155*30,      210*30,       240*30,       265*30,     310*30,     345*30]),
                       'gt_name': ['sitting', 'standing', 'walking', 'lying_d', 'lying_u', 'object_walk', 'object_pick','hand_wave', 'push_up', 'leg_exer',  'what']}

label_info[files[1]] = {'gt_id': np.int16([0,      32*30 ,    66*30,     96*30,      136*30,    196*30,     235*30,     285*30,        310*30,       360*30,   400*30]),
                       'gt_name': ['sitting', 'standing', 'lying_d', 'lying_u', 'walking', 'push_up', 'object_walk', 'object_pick', 'hand_wave', 'leg_exer', 'what']}

label_info[files[2]] = {'gt_id':np.int16([0,      25*30 ,     60*30,      90*30,     124*30,    160*30,     205*30,         245*30,       280*30,    313*30,     340*30]),
                       'gt_name': ['sitting', 'standing',  'walking', 'lying_d', 'lying_u', 'object_walk', 'object_pick', 'hand_wave', 'leg_exer', 'push_up', 'what']}

label_info[files[3]] = {'gt_id':np.int16([0,      33*30 ,    60*30,     92*30,      136*30,    180*30,        215*30,         240*30,        260*30,   280*30,   400*30]),
                       'gt_name': ['sitting', 'standing',  'walking', 'lying_d', 'lying_u', 'object_walk', 'object_pick', 'hand_wave', 'leg_exer', 'what']}

label_info[files[4]] = {'gt_id': np.int16([0,      26*30 ,     60*30,    90*30,      120*30,    150*30,     187*30,     210*30,         218*30,      240*30]),
                        'gt_name': ['sitting','standing','walking',  'lying_d', 'lying_u', 'object_walk', 'object_pick', 'hand_wave', 'leg_exer', 'what']}

label_info[files[5]] = {'gt_id': np.int16([0,       32*30 ,    60*30,     99*30,      136*30,    180*30,       220*30,          240*30,      260*30,   280*30]),
                        'gt_name': ['sitting', 'standing', 'walking', 'lying_d', 'lying_u', 'object_walk', 'object_pick', 'hand_wave', 'leg_exer', 'what']}

label_info[files[6]] = {'gt_id': np.int16([0,       34*30 ,      75*30,    115*30,    150*30,    180*30,      230*30,     270*30,     320*30,       375*30,         415*30]),
                        'gt_name':['sitting', 'standing', 'walking', 'lying_d', 'lying_u',  'object_walk', 'object_pick', 'push_up',  'leg_exer',  'hand_wave', 'what']}

label_info[files[7]] = {'gt_id': np.int16([0,      36*30 ,    76*30,     116*30,    156*30,    200*30,     270*30,        315*30,       370*30,       435*30,    460*30]),
                        'gt_name':['sitting', 'standing','walking','lying_d', 'lying_u', 'object_walk', 'object_pick', 'push_up', 'leg_exer', 'hand_wave', 'what']}

label_info[files[8]] = {'gt_id': np.int16([0,      34*30 ,      71*30,      113*30,    150*30,   196*30,      255*30,       289*30,       322*30,     340*30,   400*30]),
                        'gt_name': ['sitting', 'standing',  'walking', 'lying_d', 'lying_u', 'object_walk', 'push_up', 'object_pick', 'leg_exer', 'what']}

label_info[files[9]] = {'gt_id':  
                        np.int16([0,      25*30 ,    60*30,      78*30,      110*30,    140*30,     180*30,     205*30,        240*30,     270*30])
                        ,'gt_name': 
                        ['sitting', 'standing', 'walking', 'lying_d', 'lying_u', 'object_walk', 'push_up',  'hand_wave', 'leg_exer', 'what']
                       }

label_info[files[10]] = {'gt_id': 
                         np.int16([0,      47*30 ,    92*30,     153*30,      200*30,    250*30,      290*30,     325*30,        360*30,      400*30,   448*30])   
                         ,'gt_name': 
                         ['sitting', 'standing', 'lying_d', 'lying_u', 'walking', 'object_walk', 'object_pick', 'push_up', 'hand_wave', 'leg_exer', 'what'] 
                        }

label_info[files[11]] = {'gt_id': 
                         np.int16([0,     27*30 ,     60*30,      95*30,      125*30,   165*30,      205*30,      250*30,     280*30,       320*30,      340*30])    
                         ,'gt_name': 
                         ['sitting', 'standing', 'walking', 'lying_d', 'lying_u', 'object_walk', 'object_pick', 'push_up', 'leg_exer',  'hand_wave', 'what']     
                        }

label_info[files[12]] = {'gt_id': 
                         np.int16([0,     50*30 ,      78*30,     100*30,    140*30,   177*30,        225*30,        255*30,    296*30,    336*30 ,     356*30])   
                         ,'gt_name': 
                         ['sitting', 'standing', 'walking', 'lying_d', 'lying_u', 'object_walk', 'object_pick', 'push_up', 'leg_exer',  'hand_wave', 'what']     
                        }

label_info[files[13]] = {'gt_id': 
                         np.int16([0,      30*30 ,     60*30,      96*30,     140*30,    170*30,        220*30,        240*30,     270*30,     320*30,      365*30]) 
                         ,'gt_name': 
                         ['sitting', 'standing', 'walking', 'lying_d', 'lying_u',  'object_walk', 'object_pick', 'push_up',  'leg_exer', 'hand_wave','what']
                        }

label_info[files[14]] = {'gt_id': 
                         np.int16([0,     40*30 ,    70*30,       105*30,   164*30,     220*30,       265*30,         280*30,  330*30,      390*30,     440*30])  
                         ,'gt_name': 
                         ['sitting', 'standing', 'walking', 'lying_d', 'lying_u', 'object_walk', 'object_pick', 'push_up', 'leg_exer', 'hand_wave', 'what']    
                        }

label_info[files[15]] = {'gt_id': 
                         np.int16([0,       25*30,      60*30,     90*30,     120*30,    160*30,        187*30,         210*30,        235*30,     272*30,    292*30])   
                         ,'gt_name': 
                         ['sitting', 'standing',  'walking', 'lying_d', 'lying_u', 'object_walk',  'object_pick', 'hand_wave', 'leg_exer', 'push_up', 'what']
                        }

label_info[files[16]] = {'gt_id': 
                         np.int16([0,      32*30,      65*30,     97*30,   154*30,     210*30,        260*30,         290*30,   306*30,     332*30,      360*30])   
                         ,'gt_name': 
                         ['sitting', 'standing', 'walking', 'lying_d', 'lying_u', 'object_walk', 'object_pick', 'push_up', 'leg_exer', 'hand_wave', 'what']   
                        }

label_info[files[17]] = {'gt_id': 
                         np.int16([0,      60*30 ,     115*30,     150*30,    210*30,    270*30,     310*30,        322*30,        370*30,       399*30])             
                         ,'gt_name': 
                         ['sitting', 'walking', 'lying_u', 'lying_d',  'leg_exer',  'push_up', 'object_pick',  'object_walk',  'hand_wave',  'what']       
                        }


label_info[files[18]] = {'gt_id': 
                        np.int16([0,      39*30,      68*30,      105*30,   140*30,   180*30,        225*30,        250*30,    278*30,    310*30,       350*30])
                        ,'gt_name': 
                         ['sitting', 'standing', 'walking', 'lying_d', 'lying_u', 'object_walk', 'object_pick', 'push_up', 'leg_exer', 'hand_wave', 'what']
                        }

label_info[files[19]] = {'gt_id': 
                         np.int16([0,      27*30,     45*30,      80*30,     115*30,    150*30,       200*30,         240*30,     270*30,  310*30,       340*30])  
                         ,'gt_name': 
                         ['sitting', 'standing', 'walking', 'lying_d', 'lying_u', 'object_walk', 'object_pick', 'push_up', 'leg_exer','hand_wave',  'what']
                        }

label_info[files[20]] = {'gt_id': 
                         np.int16([0,   28*30,       54*30,       85*30,        114*30,       145*30,    188*30,     223*30,    255*30,      280*30,       313*30])  
                         ,'gt_name': 
                         ['sitting', 'standing', 'walking', 'object_walk', 'object_pick', 'lying_d', 'lying_u',  'push_up', 'leg_exer', 'hand_wave', 'what']
                        }

label_info[files[21]] = {'gt_id': 
                         np.int16([0,      25*30 ,     55*30,     95*30,      122*30,          150*30,     185*30,     220*30,     257*30,     287*30,     310*30])
                         ,'gt_name': 
                         ['sitting', 'standing', 'walking', 'object_walk', 'object_pick', 'lying_d', 'lying_u',  'push_up',  'leg_exer',  'hand_wave', 'what']
                        }

label_info[files[22]] = {'gt_id': 
                         np.int16([0,      25*30 ,    60*30,      93*30,      126*30,    160*30,      210*30,        235*30,      260*30,     300*30,     312*30])      
                         ,'gt_name': 
                         ['sitting', 'standing', 'walking', 'lying_d', 'lying_u','object_walk', 'object_pick', 'push_up',  'leg_exer', 'hand_wave',  'what']
                        }

label_info[files[23]] = {'gt_id': 
                         np.int16([0,      27*30 ,    60*30,      92*30,        120*30,        150*30,     190*30,     225*30,    250*30,     277*30,     293*30])  
                         ,'gt_name': 
                         ['sitting', 'standing', 'walking', 'object_walk', 'object_pick', 'lying_d', 'lying_u',  'push_up', 'leg_exer', 'hand_wave',  'what']
                        }

label_info[files[24]] = {'gt_id': 
                         np.int16([0,      43*30 ,    73*30,     105*30,     145*30,    176*30,       225*30,         247*30,     285*30,    320*30,       350*30]) 
                         ,'gt_name': 
                         ['sitting', 'standing', 'walking', 'lying_d', 'lying_u', 'object_walk', 'object_pick', 'push_up', 'leg_exer', 'hand_wave',  'what']
                        }

label_info[files[25]] = {'gt_id': 
                         np.int16([0,     27*30 ,    58*30,       80*30,     110*30,    140*30,       200*30,        230*30,     260*30,    325*30,      355*30])
                         ,'gt_name': 
                         ['sitting', 'standing', 'walking', 'lying_d', 'lying_u', 'object_walk', 'object_pick', 'push_up', 'leg_exer', 'hand_wave', 'what']
                        }


label_info[files[26]] = {'gt_id': 
                         np.int16([0,       28*30 ,     60*30,     93*30,         133*30,    170*30,    210*30,     230*30,      270*30,         312*30,     350*30])
                         ,'gt_name': 
                         ['sitting', 'standing', 'walking',  'object_walk', 'lying_d', 'lying_u',  'push_up',  'leg_exer', 'object_pick',  'hand_wave', 'what']
                        }

label_info[files[27]] = {'gt_id':
                         np.int16([0,      18*30 ,    75*30,      115*30,   175*30,     220*30,        265*30,         314*30,     375*30,     425*30,      455*30])  
                         ,'gt_name': 
                         ['sitting', 'standing','walking', 'lying_d', 'lying_u',  'object_pick', 'object_walk',  'push_up',  'leg_exer', 'hand_wave','what']
                        }

label_info[files[28]] = {'gt_id':
                         np.int16([0,      32*30 ,     60*30,     110*30,     132*30,    166*30,        205*30,       243*30,      268*30,    306*30,       336*30])  
                         ,'gt_name': 
                         ['sitting', 'standing', 'walking', 'lying_d', 'lying_u',  'object_pick', 'object_walk', 'push_up', 'leg_exer', 'hand_wave',  'what']   
                        }

In [ ]:
with open('label_info.pkl', 'wb') as f:
    pickle.dump(label_info, f)

In [ ]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
output.backward()



import torch.nn.functional as F
targ = F.one_hot(target, num_classes = 5)
targ = targ.to(torch.float)
target1 = torch.randn(3, 5).softmax(dim=1)
opt = loss(input, targ)